# FALLOS SCRAPING

## 1. Configuración del entorno

### 1.1. Configuración del entorno del script

In [ ]:
%pip install selenium
%pip install requests_toolbelt
%pip install beautifulsoup4
%pip install chromedriver-autoinstaller

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os, re, json, html, time
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder

### 1.2. Codificación del sistema legal

- 328 = Casos Civiles
- 270 = Casos de Familia
- 528 = Corte Suprema
- 268 = Casos Penales
- 269 = Casos de Cobranza
- 271 = Casos Laborales
- 168 = Casos Corte de Apelaciones

### 1.3. Consideraciones

Las varaibles `limit` y `offset` son importantes para la paginación de los resultados. El `limit` define cuántos resultados se obtienen por página, mientras que el `offset` indica desde qué punto comenzar a obtener los resultados. Por defecto, el `limit` es 10, pero puede ser modificado según las necesidades del scraping.

```python
limit  = 10
offset = 0
os.makedirs("fallos_txt", exist_ok=True) 
```

Si el servicio se cae, es posible reiniciar el scraping desde el último `offset` guardado. Esto permite retomar el proceso sin iniciar desde cero. Además, se crea un directorio específico para almacenar los archivos de texto de los fallos obtenidos, lo que facilita la organización y el acceso a los datos.

Para un proceso manual más ordenado, se definió hacer scraping por cada tipo de fallo, extrayendo en grupos de 10. Esto ya que se desconoce cuántos fallos se pueden obtener sin que el servicio se caiga.

## 2. Extraer Fallos Civiles

Este script extrae todos los fallos civiles de la Corte Suprema de Chile desde el sitio web oficial, guardándolos en archivos de texto. 

In [32]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_civiles_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "328", # 328 = Civiles
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_sentencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"CIVILES_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)


🗂  Página offset 0 (10 fallos)
   💾 CIVILES_130086751_2021-05-12__1º_Juzgado_de_Letras_de_Rengo__COMERCIAL_TECNOVA_LIMITADAAGROFOODS_CENTRAL_VALLEY_CHILE_SA.txt
   💾 CIVILES_16327798_2010-09-07__4º_Juzgado_de_Letras_Civil_de_Antofagasta__CASTRO_VIVANCO.txt
   💾 CIVILES_55272230_2015-04-07__Juzgado_de_Letras_de_Arauco__ALBURQUENQUE_.txt
   💾 CIVILES_165013358_2024-03-25__9º_Juzgado_Civil_de_Santiago__CISTERNASFISCO-CONSEJO_DE_DEFENSA_DEL_ESTADO.txt
   💾 CIVILES_4484050_2003-04-15__16º_Juzgado_Civil_de_Santiago__ERRAZURIZARAVENA.txt
   💾 CIVILES_113020155_2019-12-19__12º_Juzgado_Civil_de_Santiago__SOCIEDAD_COMERCIAL_E_INVERSIONES_ARZEAGRICOLA_Y_COMERCIAL_FR.txt
   💾 CIVILES_66980916_2016-04-29__2º_Juzgado_Civil_de_Puerto_Montt__FUCHSLOCHER_.txt
   💾 CIVILES_42616421_2013-11-25__1º_Juzgado_de_Letras_de_San_Antonio__GONZALEZ_.txt
   💾 CIVILES_21731397_2011-06-29__23º_Juzgado_Civil_de_Santiago__ARAVENA_ARAVENA.txt
   💾 CIVILES_127968066_2021-03-05__1º_Juzgado_Civil_de_Puente_Alto__PROMOTORA

KeyboardInterrupt: 

## 3. Extraer Fallos Familia

Este script extrae todos los fallos de familia de la Corte Suprema de Chile desde el sitio web oficial, guardándolos en archivos de texto. El response es distinto al de los fallos civiles, por lo que se debe ajustar el código para manejarlo correctamente.

In [27]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_familia_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "270",          # 270 = Familia
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_setencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"FAMILIA_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)


🗂  Página offset 0 (10 fallos)
   💾 FAMILIA_2024-12-26__Juzgado_de_Familia_Antofagasta__ANONIMIZADO.txt
   💾 FAMILIA_2023-11-02__Juzgado_de_Familia_Temuco__ANONIMIZADO.txt
   💾 FAMILIA_2024-10-25__Juzgado_de_Familia_Chillan__ANONIMIZADO.txt
   💾 FAMILIA_2024-03-21__Juzgado_de_Familia_Iquique__ANONIMIZADO.txt
   💾 FAMILIA_2023-08-14__Juzgado_de_Familia_Buin__ANONIMIZADO.txt
   💾 FAMILIA_2023-07-27__1_Juzgado_de_Familia_San_Miguel__ANONIMIZADO.txt
   💾 FAMILIA_2023-03-08__Juzgado_de_Familia_Puente_Alto__ANONIMIZADO.txt
   💾 FAMILIA_2023-07-24__Juzgado_de_Familia_Puerto_Varas__ANONIMIZADO.txt
   💾 FAMILIA_2023-02-22__Jgdo_L_y_G_de_Cochrane__ANONIMIZADO.txt
   💾 FAMILIA_2023-04-06__Jgdo_L_y_G_de_Santa_Juana__ANONIMIZADO.txt


KeyboardInterrupt: 

## 4. Extraer Fallos Penales
Este script extrae todos los fallos penales de la Corte Suprema de Chile desde el sitio web oficial, guardándolos en archivos de texto.

In [28]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_penales_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "268",          # 268 = Penales
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_sentencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"PENALES_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)


🗂  Página offset 0 (10 fallos)
   💾 PENALES_2024-12-16__Juzgado_de_Garantía_de_Puerto_Montt__MP_C_PABLO_ANTONIO_GONZÁLEZ_ALMONACID.txt
   💾 PENALES_2025-03-21__Tribunal_de_Juicio_Oral_en_lo_Penal_Viña_del_Mar__C_CHRISTIAN_ANDRÉS_ORREGO_CASTILLO.txt
   💾 PENALES_2025-02-10__Juzgado_de_Garantía_de_la_Serena__CESAR_GUSTAVO_VEGA_CASTRO_C_MARCO_ALEJANDRO_BERRÍOS_CASTRO.txt
   💾 PENALES_2025-03-07__Juzgado_de_Letras_y_Garantía_de_Alto_Hospicio__JAIME_FERER_MUÑOZ_C_FRANCISCO_JAVIER_PIÑA_PAVEZ.txt
   💾 PENALES_2025-04-21__6º_TRIBUNAL_DE_JUICIO_ORAL_EN_LO_PENAL_DE_SANTIAGO__C_JOSE_ALFREDO_HUARACHE_BRITO.txt
   💾 PENALES_2024-12-19__Juzgado_de_Letras_y_Garantía_de_Panguipulli__MP_C_OSVALDO_DAVID_DELGADO_GALLARDO.txt
   💾 PENALES_2025-02-22__3º_TRIBUNAL_DE_JUICIO_ORAL_EN_LO_PENAL_DE_SANTIAGO__C_JUAN_LUIS_EDUARDO_VARAS_ROJAS.txt
   💾 PENALES_2024-12-19__Juzgado_de_Letras_y_Garantía_de_Alto_Hospicio__FL_AH_C_ULISES_KREMLIN_INTI_TADEO.txt
   💾 PENALES_2025-01-23__Tribunal_de_Juicio_Oral_en_lo_Penal_

KeyboardInterrupt: 

## 5. Extraer Fallos Cobranza
Este script extrae todos los fallos de cobranza de la Corte Suprema de Chile desde el sitio web oficial, guardándolos en archivos de texto.

In [18]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_cobranza_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "269", # 269 = Cobranza
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_sentencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"COBRANZA_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)


🗂  Página offset 0 (10 fallos)
   💾 2025-04-30__Jdo_Cob_Laboral_y_Previsional_de_Santiago__AFP_PROVIDA_SA_CON_CAPITALES_SA.txt
   💾 2025-04-30__Jdo_Cob_Laboral_y_Previsional_de_Santiago__AFP_MODELO_SA_CON_COMUNIDAD_CENTRO_COMERCIAL_PERSA_ESTACION.txt
   💾 2025-04-30__Jdo_de_Letras_del_Trabajo_de_Temuco__AFP_MODELO_SA_CON_I_MUNICIPALIDAD_DE_TEMUCO.txt
   💾 2025-04-29__Jdo_de_Letras_del_Trabajo_de_San_Felipe__AFP_HABITAT_SA_CON_ILUSTRE_MUNICIPALIDAD_SAN_FELIPE.txt
   💾 2025-04-29__Jdo_Cob_Laboral_y_Previsional_de_Santiago__AFP_CUPRUM_SA_CON_CAPITALES_SA.txt
   💾 2025-04-29__Jdo_Cob_Laboral_y_Previsional_de_Santiago__AFP_CUPRUM_SA_CON_PROQUALITAS_CONSULTORES_LIMITA.txt
   💾 2025-04-29__Jdo_de_Letras_del_Trabajo_de_Rancagua__ADM_DE_FONDOS_DE_CESANTIA_CHILE_II_SA_CON_SERVICIO_DE_SALUDO.txt
   💾 2025-04-29__Jdo_Cob_Laboral_y_Previsional_de_Santiago__ADM_DE_FONDOS_DE_CESANTIA_CHILE_II_SA_CON_ABENGOA_CHILE_S_A.txt
   💾 2025-04-29__Jdo_Cob_Laboral_y_Previsional_de_San_Miguel__PÉREZILUSTRE_MUNIC

## 6. Extraer Fallos Laborales

Este script extrae todos los fallos laborales de la Corte Suprema de Chile desde el sitio web oficial, guardándolos en archivos de texto.

In [29]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_laborales_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "271", # 271 = Laborales
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_sentencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"LABORALES_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)


🗂  Página offset 0 (10 fallos)
   💾 LABORALES_2024-05-17__Juzgado_de_Letras_de_Los_Lagos__HERRERAGENERA_CONTACT_CENTER_SPA.txt
   💾 LABORALES_2024-11-05__1º_Juzgado_de_Letras_del_Trabajo_de_Santiago__FIGUEROACOMERCIAL_ECCSA_SA.txt
   💾 LABORALES_2024-04-11__Juzgado_de_Letras_y_Garantía_de_Freirina__MUÑOZCONSORCIO_VICCSA_GEVIAL_LTDA.txt
   💾 LABORALES_2024-12-26__1º_Juzgado_de_Letras_del_Trabajo_de_Santiago__LEÓNABENIS_CONSULTORES_SPA.txt
   💾 LABORALES_2015-05-14__2º_Juzgado_de_Letras_del_Trabajo_de_Santiago__CAYUPI_CON_DEL_SUR_SA.txt
   💾 LABORALES_2011-08-11__2º_Juzgado_de_Letras_del_Trabajo_de_Santiago__SILVA_CON_EMERGIA_CONTACT_CENTER_SL_CHILE_LIMITADA.txt
   💾 LABORALES_2025-02-20__2º_Juzgado_de_Letras_del_Trabajo_de_Santiago__SODEXO_CHILE_SACENTRO_DE_CONCILIACIÓN_Y_MEDIACIÓN_METROPOLIT.txt
   💾 LABORALES_2019-02-20__Juzgado_de_Letras_del_Trabajo_de_Castro__FUNDACIÓN_ALMIRANTE_CARLOS_CONDELLINSPECCION_PROVINCIAL_DEL_.txt
   💾 LABORALES_2022-07-08__2º_Juzgado_de_Letras_del_Trabajo_

## 7. Extraer Fallos Corte de Apelaciones

Este script extrae todos los fallos de la Corte de Apelaciones de Chile desde el sitio web oficial, guardándolos en archivos de texto.

In [30]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_apelaciones_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "168", # 168 = Corte de Apelaciones
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_sentencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"APELACIONES_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)


🗂  Página offset 0 (10 fallos)
   💾 APELACIONES_2016-07-11__JUZGADO_DE_LETRAS_DEL_TRABAJO_DE_VALPARAISO__FLORES_CON_CONSTRUCTORA_ALTIUS_SA.txt
   💾 APELACIONES_2016-07-19__JUZGADO_DE_LETRAS_Y_GARANTIA_DE_CURACAUTIN__MARCHANT_HERNANDEZ_EMA_CON_MARCHANT_VALLEJOS_FELIPE_Y_OTROS.txt
   💾 APELACIONES_2016-07-25__2º_JUZGADO_DE_LETRAS_DE_TALAGANTE__PROMOTORA_CMR_FALABELLA_SA_CON_ESPINOZA_JORQUERA_MARIA_JOSE.txt
   💾 APELACIONES_2016-07-11__5º_JUZGADO_CIVIL_DE_SANTIAGO__CADEMARTORI_GAMBOA_DÍAZ_LEAL.txt
   💾 APELACIONES_2021-10-04__SIN_INFORMACION__ORTIZDEPARTAMENTO_DE_EXTRANJERÍA_Y_MIGRACIÓN_DEL_MINISTERIO_.txt
   💾 APELACIONES_2016-07-08__JUZGADO_DE_LETRAS_DEL_TRABAJO_DE_PUENTE_ALTO__NAVARRETE_CON_COMPAÑIA_INDUSTRIAL_EL_VOLCAN_SA.txt
   💾 APELACIONES_2016-07-08__TRIBUNAL_TESORERIA_PROVINCIAL_CONCEPCION__FISCO_CON_DEUDORES_MOROSOS.txt
   💾 APELACIONES_2021-10-04__SIN_INFORMACION__FERNÁNDEZRÍOS.txt
   💾 APELACIONES_2022-10-26__SIN_INFORMACION__URBINA_COMISIÓN_DE_LIBERTAD_CONDICIONAL_DE_LA_REGIÓ

## 8. Extraer Fallos Corte Suprema

Este script extrae todos los fallos de la Corte Suprema de Chile desde el sitio web oficial, guardándolos en archivos de texto.

In [31]:
opt = webdriver.ChromeOptions()
opt.add_argument("--headless"); opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
dr = webdriver.Chrome(options=opt)

dr.get("https://juris.pjud.cl/busqueda?Corte_Suprema")
dr.implicitly_wait(5)

cookies = {c["name"]: c["value"] for c in dr.get_cookies()}
xsrf_token = cookies.get("XSRF-TOKEN", "")

soup = BeautifulSoup(dr.page_source, "html.parser")
form_token = soup.find("input", {"name": "_token"})["value"]
dr.quit()

def clean_filename(text):
    """Convierte texto a algo seguro como nombre de archivo"""
    text = re.sub(r"[^\w\s-]", "", text, flags=re.U)
    return re.sub(r"\s+", "_", text)[:60]

def html_to_text(raw_html: str) -> str:
    """Quita <br>, etiquetas y decodifica entidades"""
    raw_html = raw_html.replace("<br/>", "\n").replace("<br />", "\n").replace("<br>", "\n")
    text = re.sub(r"<[^>]+>", "", raw_html)
    return html.unescape(text)

# ---------- IMPORTANTE: paginación sobre buscar_sentencias ----------
limit  = 10          # por defecto, 10 fallos por página
offset = 0           # inicia en 0 y va incrementando de a 10. Si se cae el servicio, se puede reiniciar desde el último offset
os.makedirs("fallos_suprema_txt", exist_ok=True)

session = requests.Session()
headers = {
    "X-XSRF-TOKEN": xsrf_token,
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent": "Mozilla/5.0"
}

while True:
    m = MultipartEncoder(
        fields={
            "_token": form_token,
            "id_buscador": "528", # 528 = Corte Suprema
            "filtros": json.dumps({        # SIN filtros
                "facetas_seleccionadas": [],
                "filtros_omnibox": [{"categoria": "TEXTO", "valores": [""]}],
            }),
            "numero_filas_paginacion": str(limit),
            "offset_paginacion": str(offset),
            "orden": "rel",
            "personalizacion": "false"
        }
    )
    headers["Content-Type"] = m.content_type

    r = session.post("https://juris.pjud.cl/busqueda/buscar_sentencias",
                     headers=headers, cookies=cookies, data=m)

    if r.status_code != 200:
        print(f"❌ HTTP {r.status_code} en offset {offset}")
        break

    data = r.json()
    docs = data.get("response", {}).get("docs", [])
    if not docs:
        print("✅ Fin de resultados")
        break

    print(f"🗂  Página offset {offset} ({len(docs)} fallos)")

    for d in docs:
        id_fallo = d.get("id", "")
        cuerpo_raw = d.get("texto_sentencia") or ""
        if not cuerpo_raw:
            continue

        cuerpo_txt = html_to_text(cuerpo_raw).strip()

        fecha = (d.get("fec_sentencia_sup_dt") or "").split("T")[0] or "sin_fecha"
        juzgado = clean_filename(d.get("gls_juz_s", "desconocido"))
        carat   = clean_filename(d.get("caratulado_s", "sin_caratula"))

        filename = f"SUPREMA_{id_fallo}_{fecha}__{juzgado}__{carat}.txt"
        path = os.path.join("fallos_txt", filename)

        with open(path, "w", encoding="utf-8") as f:
            f.write(cuerpo_txt)

        print("   💾", filename)

    offset += limit
    time.sleep(2)


🗂  Página offset 0 (10 fallos)
   💾 SUPREMA_2013-12-18__desconocido__ACUÑA_VIVALLOS_MARIA_ANGELICA_Y_OTROS_CONTRA_RES_EXENTA_N_21.txt
   💾 SUPREMA_2013-12-18__3º_JUZGADO_DE_LETRAS_DE_SAN_BERNARDO__TRANSPORTES_REFRIGERADOS_JORGE_FERNANDEZ_SA_CON_VARGAS_BARRI.txt
   💾 SUPREMA_2013-12-18__2º_JUZGADO_DE_LETRAS_DE_IQUIQUE__JIMENEZ_MARABOLI_MARIA_CON_CONSORCIO_PESQUERO_DEL_NORTE.txt
   💾 SUPREMA_2013-12-17__desconocido__NICOLAS_IGNACIO_BLANCO_GAJARDO_CONTRA_PREFECTO_DE_CARABINERO.txt
   💾 SUPREMA_2013-12-17__2º_JUZGADO_DEL_CRIMEN_DE_SANTIAGO__ANONIMIZADO.txt
   💾 SUPREMA_2013-12-17__TRIBUNAL_DE_JUICIO_ORAL_EN_LO_PENAL_DE_TALCA__ANONIMIZADO.txt
   💾 SUPREMA_2013-12-17__2º_JUZGADO_POLICIA_LOCAL_DE_PROVIDENCIA__SERNAC_CON_TIME_FOR_FUN_CHILE_SA.txt
   💾 SUPREMA_2013-12-17__JUZGADO_DE_GARANTIA_DE_CHILLAN__ANONIMIZADO.txt
   💾 SUPREMA_2013-12-17__desconocido__JAEL_PAMELA_MARAMBIO_HERNANDEZ_RECURRIDO_DIRECTOR_HOSPITAL_R.txt
   💾 SUPREMA_2013-12-17__desconocido__ANONIMIZADO.txt
